In [12]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections
from pyspark.ml.linalg import Vectors
from sklearn.cluster import MiniBatchKMeans, KMeans
import numpy as np

In [13]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "C:/temp").appName("SparkSQL").getOrCreate()

In [14]:
inputLines = spark.sparkContext.textFile("test_full_parsed_clean2.csv")

In [15]:
header=inputLines.filter(lambda l: "_id" in l)

In [16]:
header.collect()

['id,log_duration,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,distance,snow,holiday,vendor_id,pickup_hour_0,pickup_hour_1,pickup_hour_2,pickup_hour_3,pickup_hour_4,pickup_hour_5,pickup_hour_6,pickup_hour_7,pickup_hour_8,pickup_hour_9,pickup_hour_10,pickup_hour_11,pickup_hour_12,pickup_hour_13,pickup_hour_14,pickup_hour_15,pickup_hour_16,pickup_hour_17,pickup_hour_18,pickup_hour_19,pickup_hour_20,pickup_hour_21,pickup_hour_22,pickup_hour_23,pickup_weekday_0,pickup_weekday_1,pickup_weekday_2,pickup_weekday_3,pickup_weekday_4,pickup_weekday_5,pickup_weekday_6,pickup_month_1,pickup_month_2,pickup_month_3,pickup_month_4,pickup_month_5,pickup_month_6,passenger_count_0,passenger_count_1,passenger_count_2,passenger_count_3,passenger_count_4,passenger_count_5,passenger_count_6,passenger_count_7,route_coordinates']

In [17]:
linesnoheader = inputLines.subtract(header)

In [18]:
linesnoheader.take(2)

['id0232003,6.093569770045136,-73.99506378173827,40.75955581665039,-73.98538970947266,40.77353286743164,1755.2674031046058,2347.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,"[[-73.995121, 40.759478], [-73.995473, 40.759626], [-73.988561, 40.769118], [-73.991402, 40.770319], [-73.988185, 40.774749], [-73.985374, 40.773555]]"',
 'id0452714,5.748437115196633,-73.96681976318358,40.79372787475585,-73.97754669189453,40.78790664672852,1111.3896770854244,1482.3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,"[[-73.966875, 40.793751], [-73.972674, 40.785804], [-73.977582, 40.787861]]"']

In [19]:
coord=pd.read_csv('route_coord_count.csv')
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000).fit(coord[['longitude','latitude']])
coord.loc[:, 'cluster'] = kmeans.predict(coord[['longitude','latitude']])

In [20]:
cluster=coord['cluster'].tolist()

cluster_name=['C'+str(i) for i in range(100)]


In [44]:
def topcoordinates(line):
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    
    line_cluster=[]
    for i in coordinates:
            temp=re.split('\[|, |\]',i)
            long=float(temp[1])
            lat=float(temp[2])
            cluster_group=kmeans.predict([[long,lat]])
            line_cluster+=[cluster_group[0]]

    line_cluster=list(set(line_cluster))
    all_cluster=[ 1 if i in line_cluster else 0 for i in range(100)]
        
    
    
    
    feature=line.split(',')
    feature_id=[feature[0]]
    feature1=[float(i) for i in feature[1:56]]
    feature2=[int(i) for i in feature1[7:55]]
    feature3=feature1[:7]
    return feature_id+feature3+feature2+all_cluster

    

In [40]:
a='id0232003,6.093569770045136,-73.99506378173827,40.75955581665039,-73.98538970947266,40.77353286743164,1755.2674031046058,2347.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,"[[-73.995121, 40.759478], [-73.995473, 40.759626], [-73.988561, 40.769118], [-73.991402, 40.770319], [-73.988185, 40.774749], [-73.985374, 40.773555]]"'
coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',a)
line_cluster=[]
for i in coordinates:
        temp=re.split('\[|, |\]',i)
        long=float(temp[1])
        lat=float(temp[2])
        cluster_group=kmeans.predict([[long,lat]])
        line_cluster+=[cluster_group[0]]
        
line_cluster=list(set(line_cluster))
all_cluster=[ 1 if i in line_cluster else 0 for i in range(100)]

In [45]:
coord = linesnoheader.map(topcoordinates)

In [23]:
feature=['id','log_duration', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'great_circle_distance','distance',
       'snow', 'holiday', 'vendor_id', 'pickup_hour_0',
       'pickup_hour_1', 'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4',
       'pickup_hour_5', 'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8',
       'pickup_hour_9', 'pickup_hour_10', 'pickup_hour_11',
       'pickup_hour_12', 'pickup_hour_13', 'pickup_hour_14',
       'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20',
       'pickup_hour_21', 'pickup_hour_22', 'pickup_hour_23',
       'pickup_weekday_0', 'pickup_weekday_1', 'pickup_weekday_2',
       'pickup_weekday_3', 'pickup_weekday_4', 'pickup_weekday_5',
       'pickup_weekday_6', 'pickup_month_1', 'pickup_month_2',
       'pickup_month_3', 'pickup_month_4', 'pickup_month_5',
       'pickup_month_6', 'passenger_count_0', 'passenger_count_1',
       'passenger_count_2', 'passenger_count_3', 'passenger_count_4',
       'passenger_count_5', 'passenger_count_6', 'passenger_count_7']
colNames=feature+cluster_name

In [36]:
len(colNames)

156

In [46]:
train = coord.toDF(colNames)

In [ ]:
train.write.csv("test_full_parsed_clean3.csv",header = 'true')

In [ ]:
train.repartition(1).write.format('com.databricks.spark.csv').save('path+my.csv',header = 'true')

In [47]:
train.write.parquet('parquet_cluster_18_files_testdata')

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder

In [ ]:
feature_col=colNames[0:1]+colNames[2:]
vecAssembler = VectorAssembler(inputCols=feature_col, outputCol="features")


In [ ]:
(trainingData, testData) = train.randomSplit([0.8, 0.2],seed=42)

In [ ]:

gbt = GBTRegressor(featuresCol="features",labelCol="log_trip_duration",maxIter=10,maxDepth=10, seed=42)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[vecAssembler,gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

In [ ]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils

model = GradientBoostedTrees.trainRegressor(trainingData,
                                             categoricalFeaturesInfo={}, numIterations=3)